In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

from keras.layers import Conv2D, MaxPooling2D

from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [2]:
# MNIST 데이터셋 불러오기
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [4]:
# 60000, 가로, 세로, 필터
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [5]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
print(y_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [6]:
input_shape = (28, 28, 1)     # 행, 열, 채널

model = Sequential()

# 커널 수 : 32,  커널 사이즈, 패딩 : 2  ==> 28 * 28 * 32
model.add(Conv2D(32, kernel_size=(5, 5), padding='same',input_shape = input_shape, activation='relu'))
# 14 * 14 * 32
model.add(MaxPooling2D(pool_size=(2, 2)))

# 커널 수 : 64,  커널 사이즈, 패딩 : 2  ==> 14 * 14 * 64
model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
# 7 * 7 * 64
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # 3136

model.add(Dense(1000, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 1000)              3

In [7]:
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.SGD(0.001), metrics=['accuracy'])
early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3)

In [8]:
# fit() 메서드로 모델 훈련 시키기
hist = model.fit(
    x_train, y_train,
    epochs = 20, batch_size = 256,
    validation_data=(x_test, y_test),
    callbacks = early_stopping
    )

Epoch 1/20
235/235 [==============================] - 10s 14ms/step - loss: 1.2065 - accuracy: 0.8371 - val_loss: 0.1944 - val_accuracy: 0.9391
Epoch 2/20
235/235 [==============================] - 2s 11ms/step - loss: 0.1603 - accuracy: 0.9500 - val_loss: 0.1276 - val_accuracy: 0.9608
Epoch 3/20
235/235 [==============================] - 2s 10ms/step - loss: 0.1180 - accuracy: 0.9633 - val_loss: 0.1175 - val_accuracy: 0.9629
Epoch 4/20
235/235 [==============================] - 2s 10ms/step - loss: 0.0963 - accuracy: 0.9700 - val_loss: 0.0923 - val_accuracy: 0.9716
Epoch 5/20
235/235 [==============================] - 2s 10ms/step - loss: 0.0830 - accuracy: 0.9750 - val_loss: 0.0883 - val_accuracy: 0.9712
Epoch 6/20
235/235 [==============================] - 2s 10ms/step - loss: 0.0728 - accuracy: 0.9779 - val_loss: 0.0812 - val_accuracy: 0.9738
Epoch 7/20
235/235 [==============================] - 3s 12ms/step - loss: 0.0656 - accuracy: 0.9805 - val_loss: 0.0769 - val_accuracy: 0.976

In [9]:
# 테스트 데이터로 정확도 측정하기
test_loss, test_acc = model.evaluate(x_test, y_test)
print('test_acc: ', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0538 - accuracy: 0.9835
test_acc:  0.9835000038146973


In [10]:
predit = model.predict(x_test)
#for i in range(0,len(predit)) :
#    print("예측값 : ", predit[i], "\t실제값 : ", test_Y[i])
print(predit[0:5])
print(y_test[0 :5])

313/313 [==============================] - 1s 2ms/step
[[3.9371546e-09 2.5131217e-09 2.1555999e-07 1.1664404e-06 8.8387082e-11
  6.7312565e-11 1.4523354e-12 9.9999857e-01 7.7791896e-13 1.0170468e-08]
 [3.2927369e-10 4.3505852e-06 9.9999559e-01 9.0420489e-09 9.9548069e-14
  4.3812493e-12 1.2614180e-07 2.4707998e-12 1.4239350e-10 6.4356148e-13]
 [2.3198309e-07 9.9995470e-01 6.7123174e-06 1.1572532e-09 7.2991293e-06
  1.2861908e-07 1.1018114e-06 2.9411092e-05 3.9900891e-07 7.0378164e-08]
 [9.9999070e-01 7.8246680e-12 2.4830740e-06 5.6839560e-09 8.0039869e-10
  1.0556914e-07 6.4666319e-06 6.9467326e-10 3.3088079e-09 1.8567957e-07]
 [1.3096755e-11 2.0737843e-09 7.2543577e-10 4.5718987e-10 9.9999619e-01
  3.3652701e-12 7.1444410e-09 1.4341019e-07 6.4637469e-11 3.7513855e-06]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
